In [2]:
from tweet_processor import TweetProcessor
from word_embedding_model import WordEmbeddingModel
from sentence_embedding_model import SentenceEmbeddingModel
import dataset_handler
# from hashtag_to_sentence_space_mapper import HashtagToSentenceSpaceMapper

dataset_file_path = 'datasets/hashtags-en-tweets.jsonl'

tweet_processor = TweetProcessor()

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
word_emb_tweets = tweet_processor.preprocess_tweets_for_word_embedding(dataset_file_path)

: 

In [4]:
word_embedding_model = WordEmbeddingModel()
word_embedding_model.train(word_emb_tweets)

In [5]:
sent_emb_tweets = tweet_processor.preprocess_tweets_for_sentence_embedding(dataset_file_path)

In [8]:
train_tweets, test_tweets = dataset_handler.split_into_train_test(sent_emb_tweets)

sentences_train, sentences_test, targets_train, targets_test, test_hashtags = dataset_handler.prepare_model_inputs_and_targets(
    train_tweets, test_tweets, word_embedding_model
)

targets_train.shape, targets_test.shape

some hashtags were filtered out since they are not contained in the Word2Vec-Vocab tweet-hashtags: ['#HistoryShots', '#PolandIn', '#WorldIn'] | hashtags after filtering: []
some hashtags were filtered out since they are not contained in the Word2Vec-Vocab tweet-hashtags: ['#achizitii', '#contractari', '#procurement'] | hashtags after filtering: ['#procurement']
some hashtags were filtered out since they are not contained in the Word2Vec-Vocab tweet-hashtags: ['#Thesixthland', '#IBCFacts', '#IBCTamil'] | hashtags after filtering: ['#IBCTamil']
some hashtags were filtered out since they are not contained in the Word2Vec-Vocab tweet-hashtags: ['#soul', '#groove', '#dub'] | hashtags after filtering: ['#soul', '#dub']
some hashtags were filtered out since they are not contained in the Word2Vec-Vocab tweet-hashtags: ['#nationalhospitalityday', '#eatdrinkdochill'] | hashtags after filtering: ['#eatdrinkdochill']
some hashtags were filtered out since they are not contained in the Word2Vec-Voca

((133306,), (44369,))

In [1]:
sent_emb_model = SentenceEmbeddingModel()
sent_embs_train = sent_emb_model.generate_embeddings(sentences_train)
sent_embs_test = sent_emb_model.generate_embeddings(sentences_test)
sent_embs_train.shape, sent_embs_test.shape

NameError: name 'SentenceEmbeddingModel' is not defined

In [ ]:
from hashtag_to_sent_mapper import Hashtag2SentMapper
import pytorch_lightning as pl
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

hashtag2SentMapper = Hashtag2SentMapper(targets_train.shape[1], sent_embs_train.shape[1]).to(device) # TODO shape richtig?

trainer = pl.Trainer(gpus=1, max_epochs=50)
                            # callbacks=[EarlyStopping(monitor='val_loss', patience=100)]) 
trainer.fit(hashtag2SentMapper, data)
trainer.test()